# Description of VGG16 model
VGG16 (also called OxfordNet) is a convolutional neural network architecture named after the Visual Geometry Group from Oxford, who developed it.
It was used to win the ILSVR (ImageNet) competition in 2014.
To this day is it still considered to be an excellent vision model, although it has been somewhat outperformed by more revent advances such as Inception and ResNet.


In [1]:
import os

In [2]:
original_dataset_dir = "train"
base_dir = 'cats_and_dogs_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')
print(train_dir)
print(validation_dir)
print(test_dir)

In [3]:
from tensorflow.keras.applications import VGG16
conv_base = VGG16(weights='imagenet',include_top=False,
                  input_shape=(150, 150, 3))

In [4]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


# freezing the convolutional base
In Keras, you freeze a network by setting its trainable attribute to False:

In [5]:
print('This is the number of trainable weights ''before freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights before freezing the conv base: 30


In [6]:
conv_base.trainable = False
print('This is the number of trainable weights ''after freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights after freezing the conv base: 4


# Training the model end to end with a frozen convolutional base

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                rescale=1./255,
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
train_generator = train_datagen.flow_from_directory(
                    train_dir,
                    target_size=(150, 150),
                    batch_size=20,
                    class_mode='binary')

Found 2000 images belonging to 2 classes.


In [9]:
validation_generator = test_datagen.flow_from_directory(
                        validation_dir,
                        target_size=(150, 150),
                        batch_size=20,
                        class_mode='binary')

Found 1000 images belonging to 2 classes.


In [10]:
from tensorflow.keras import optimizers
model.compile(loss='binary_crossentropy',
                optimizer=optimizers.RMSprop(lr=2e-5),
                metrics=['acc'])

In [12]:
history = model.fit_generator(
                train_generator,
                steps_per_epoch=2, #100
                epochs=30,
                validation_data=validation_generator,
                validation_steps=50)

Epoch 1/30
2/2 [==============================] - 16s 8s/step - loss: 0.6881 - acc: 0.5500 - val_loss: 0.6453 - val_acc: 0.6350
Epoch 2/30
2/2 [==============================] - 15s 8s/step - loss: 0.6567 - acc: 0.6500 - val_loss: 0.6375 - val_acc: 0.6380
Epoch 3/30
2/2 [==============================] - 15s 8s/step - loss: 0.6481 - acc: 0.6000 - val_loss: 0.6289 - val_acc: 0.6620
Epoch 4/30
2/2 [==============================] - 15s 8s/step - loss: 0.6966 - acc: 0.5750 - val_loss: 0.6213 - val_acc: 0.6870
Epoch 5/30
2/2 [==============================] - 16s 8s/step - loss: 0.5728 - acc: 0.7750 - val_loss: 0.6185 - val_acc: 0.6570
Epoch 6/30
2/2 [==============================] - 15s 8s/step - loss: 0.6249 - acc: 0.6500 - val_loss: 0.6131 - val_acc: 0.6810
Epoch 7/30
2/2 [==============================] - 15s 8s/step - loss: 0.6441 - acc: 0.6500 - val_loss: 0.6013 - val_acc: 0.7040
Epoch 8/30
2/2 [==============================] - 15s 8s/step - loss: 0.6722 - acc: 0.6000 - val_loss: 0